# AIモデル訓練サンプル
PythonでEng-Fraサンプルデータを用いて，FrenchからEnglishへ翻訳するモデルを訓練する。
本モデルでは、単語単位ではなく、文字単位で分割し、学習します。

## プログラム説明

word_model.py

内部で，pytorchのtransformerライブラリを呼び出している。また，入出力の文字列の分割は，NLTKライブラリを用いて，単語単位に分割している。
各パラメタの詳細は [Pytorch](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html) を参照のこと。

- emb_size    
  単語のエンベッドサイズ
- nhead    
  transformerのMultiHeadAttentionのヘッド数
- ffn_hid_dim       
  FeedForwardNeuralNetworkの次元数
- batch_size     
  ミニバッチサイズ。メモリが足りないときや計算速度を早めたいときにはこのサイズを変更する
- num_encoder_layers    
  エンコーダ内のサブエンコーダ層の数 
- num_decoder_layers    
  デコーダ内のサブデコーダ層の数
- lr   
  学習率
- dropout    
  ドロップアウトの割合，1=100%
- num_epochs    
  学習用データを何周学習するか
- device    
  cuda: Cudaが使えるマシンではこれを選択
  mps: Apple Silicornが使えるマシンではこれを選択
  cpu: CPUで計算
- earlystop_patient    
  num_epochs以下でも，開発用データで，Lossが下がらなくなった回数がearlystop_patientより大きくなると，計算を終了させる
- output_dir    
  学習したモデルを格納するディレクトリ。ディレクトリには checkpoint_xxx.pt（xxxはepoch数）とcheckpoint_best.ptが作成され，valid lossが最も小さくなったepoch回のモデルをcheckpoint_best.ptとして保存する
- tensorboard_logdir    
  tensorboard のログを格納するディレクトリ。学習結果などを視覚化して表示できる。 tensorboard --logdir tensorboard_logdir で起動し，http://localhost:6006でアクセスすると表示される
- prefix    
  jsonl形式の訓練データ及び開発データのprefix
- source_lang    
  jsonl形式の訓練データ及び開発データでの，翻訳元となるデータにつけるキー
- target_lang     
  jsonl形式の訓練データ及び開発データでの，翻訳先となるデータにつけるキー

In [1]:
# Pythonで訓練をする
# deviceはcuda or cpu or mps
#  mps: apple silicon
# out of memoryが発生した際には、batch_sizeを減らす


# 途中から計算するときには，modelディレクトリに checkpoint_xxx.pt(xxxは計算済みのepoch数)とcheckpoint_best.pt が存在すること

!python ./word_model.py \
  --emb_size 1024 \
  --nhead 8 \
  --ffn_hid_dim 2048 \
  --batch_size 32 \
  --num_encoder_layers 12 \
  --num_decoder_layers 12 \
  --lr 0.00002 \
  --dropout 0.3 \
  --num_epochs 100 \
  --device cuda \
  --earlystop_patient 3 \
  --output_dir model \
  --tensorboard_logdir logs \
  --prefix translation \
  --source_lang fra \
  --target_lang eng \
  --train_file ../dataset/train.jsonl \
  --valid_file ../dataset/val.jsonl

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/analysis01/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/analysis01/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
num_epochs:100
Epoch: 1, Train loss: 5.722, Val loss: 8.062, Epoch time = 1208.490s
Epoch: 2, Train loss: 5.106, Val loss: 10.159, Epoch time = 1209.342s
Epoch: 3, Train loss: 4.899, Val loss: 9.065, Epoch time = 1205.414s
Epoch: 4, Train loss: 4.672, Val loss: 7.804, Epoch time = 1210.727s
Epoch: 5, Train loss: 4.561, Val loss: 6.362, Epoch time = 1208.014s
Epoch: 6, Train loss: 4.488, Val loss: 5.876, Epoch time = 1209.898s
Epoch: 7, Train loss: 4.410, Val loss: 5.306, Epoch time = 1206.584s
Epoch: 8, Train loss: 4.252, Val loss: 4.655, Epoch time = 1215.643s
Epoch: 9, Train loss: 4.078, Val loss: 4.421, Epoch time = 1205.246s
Epoch: 10, Train loss: 3.989, Val loss: 4.280, Epoch time = 1214.007s
Epoch

In [2]:
# javaのDJLで使えるようにモデルファイルを変換する

In [4]:
%load_ext tensorboard
%tensorboard --logdir logs

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/analysis01/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/analysis01/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Reusing TensorBoard on port 6006 (pid 435360), started 0:00:04 ago. (Use '!kill 435360' to kill it.)

## javaのDJLで使えるようにモデルファイルを変換する

In [3]:
!python convert.py  \
    --model_file=model/checkpoint_best.pt \
    --model_script=model/script.pt \
    --encoder=model/encoder.pt \
    --decoder=model/decoder.pt \
    --positional_encoding=model/positional_encoding.pt \
    --generator=model/generator.pt \
    --src_tok_emb=model/src_tok_emb.pt \
    --tgt_tok_emb=model/tgt_tok_emb.pt \
    --vocab_src=model/vocab_src.txt \
    --vocab_tgt=model/vocab_tgt.txt \
    --params=model/params.json \
    --device=cpu


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/analysis01/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/analysis01/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
